In [ ]:
import telebot as tb
import requests
from random import random
from datetime import datetime, timezone
def now():
    return datetime.now(timezone.utc)

import urllib.request
import ssl
import json
import time

In [ ]:
token="xxxxxxxxxx"
bot=tb.TeleBot(token)

api_key = "lunar crush api key"
#https://api.lunarcrush.com

In [ ]:
with open('data/messages.txt') as json_file:
    data = json.load(json_file)
startmessage=data['startmessage'] #message for bot start
helpmessage=data['helpmessage'] #message for /help
sorrymessage=data['sorrymessage'] #message when recieve a non-command
cryptoargs=data['cryptoargs'] #all of the arguments for /crypto
nocoin=data['nocoin'] #message for when coin couldn't be found
del data

with open('data/userinfo.txt') as json_file: #load user data
    userinfo=json.load(json_file)

with open('data/coinstoname.txt') as json_file: #load conversion 
    coinstonames=json.load(json_file)

with open('data/namestocoin.txt') as json_file: #load inverse conversion
    namestocoins=json.load(json_file)

In [ ]:
def nameofvar(var): #just to update files without ifs (lazy coding)
    return [k for k, v in globals().items() if v == var][0] 

def random_num(user): #test command just to check if bot is working
    bot.send_message(user, random())
    return

def start_bot(user): #print start message on bot start
    bot.send_message(user, startmessage)
    return

def help_bot(user): #print all commands and their explanation
    bot.send_message(user, helpmessage)
    return

def bug_rep(report): #add bugreport to file
    print(report)
    with open("data/bugreports.txt") as reports:
        reps=reports.read()
    with open("data/bugreports.txt", 'w') as reports:
        reports.write(reps+'\n'+report)
    return

def cointoname(user, args): #return the name of crypto from the sign for it
    global coinstonames
    coins=list(map(lambda x: x.lower(), args)) #ignoring capitalization
    if len(coins)==0:
        bot.send_message(user, "Command requires at least one argument")
        return
    tmp=""
    for coin in coins:
        if coin in coinstonames:
            tmp+=f"{coin} stands for {coinstonames[coin]}\n"
    bot.send_message(user, tmp.strip() if tmp!="" 
                     else f"Couldn't find a name for {' ,'.join(coins)}.")
    return

def nametocoin(user, args): #return the sign of crypto from its name
    global namestocoins
    names=(" ".join(args).lower()).split(';')
    if len(names)==0:
        bot.send_message(user, "Command requires at least one argument")
        return
    tmp=""
    for name in names:
        if name in namestocoins:
            tmp+=f"The symbol for {name} is {namestocoins[name]}\n"
    bot.send_message(user, tmp.strip() if tmp!="" else f"Couldn't find a symbol for {names}.")
    return

def getall(user, args = ["both"]):
    if len(args) > 1:
        bot.send_message(user, "This commands takes 1 or 0 arguments")
        return
    if "names" in args:
        tmp = list(coinstonames.values())
#     elif "coins" in args:
    else:
        tmp = list(namestocoins.values())
#     elif "ntc" in args:
#         tmp = list(map(lambda x: x+": "+namestocoins[x.lower()], list(coinstonames.values())))
#     else:
#         tmp = list(map(lambda x: x+": "+coinstonames[x.lower()], list(namestocoins.values())))
    j=0
    for i in range(100, 4001, 100):
        t=tmp[j:i]
        if len(t) == 0:
            break
        bot.send_message(user, "\n".join(t))
        j=i
    return

def crypto_args(user): #send all arguments that can be used with /crypto
    bot.send_message(user, cryptoargs)
    return

def updateson(var): #resave variable to text file
    with open(r"data/" + nameofvar(var)+'.txt', 'w') as json_file:
        json.dump(var, json_file)

def ping(url="http://lunarcrush.com"): #check if lunarcrush (or other service) is down
    try:
        requests.get(url)
        return True
    except:
        return False

def registertracks(user, args): #make args followed crypto for user
    if str(user) not in userinfo:
        userinfo[str(user)]={} #create register if user not already in the database
    userinfo[str(user)]["tracks"]=args
    bot.send_message(user, f"Your track list is {userinfo[str(user)]['tracks']}")
    updateson(userinfo)
    return

def addtracks(user, args): #add args to the list of followed crypto for user, if not in base, add user
    try:
        userinfo[str(user)]['tracks']=list(set(userinfo[str(user)]['tracks']) | set(args))
        bot.send_message(user, f"Your new track list is {userinfo[str(user)]['tracks']}")
        updateson(userinfo)
    except:
        registertracks(user, args)
    return

def removetracks(user, args): #remove listed crypto from list of tracks
    try:
        userinfo[str(user)]['tracks']=list(set(userinfo[str(user)]["tracks"]) - set(args))
        bot.send_message(user, f"Your new track list is {userinfo[str(user)]['tracks']}")
        updateson(userinfo)
    except:
        bot.send_message(user, "You don't have any tracks registered")
    return

def setpreference(user, args): #choose if you prefer info from geckocoin or lunarcrush
    if len(args)>1 or len(args)==0:
        bot.send_message(user, "Command takes exactly 1 argument")
    if str(user) not in userinfo:
        userinfo[str(user)]={}
    if args[0]!='geckocoin' and args[0]!='lunarcrush':
        bot.send_message(user, "Argument must be geckocoin or lunarcrush")
        return
    userinfo[str(user)]['preference']=args[0]
    bot.send_message(user, f"Your preference is now set to {userinfo[str(user)]['preference']}")
    updateson(userinfo)
    return

def gettracks(user): #get a list of your followed crypto
    try:
        bot.send_message(user, f"Your current track list is {userinfo[str(user)]['tracks']}")
    except:
        bot.send_message(user, "You have no registered tracks")
    return

def track_updates(user, args): #get updates for your followed tracks. basically shortcut for /crypto
    if str(user) not in userinfo:
        bot.send_message(user, "Register your tracks to use this command")
#         print(user)
        return
    
    for coin in userinfo[str(user)]['tracks']:
        crypto_bot(user, [coin, *args])
    return

def crypto_bot(user, args): #the main function. get info about crypto.
    if len(args)==0:
        bot.send_message(user, "/crypto must have 1 argument and any options")
        return
    coin=args[0]
    reqargs=args[1:]
    pref = "lunarcrush"
    try: #check if user has a registered preference in the database
        pref=userinfo[str(user)]['preference']
    except:
        pass
                
    if "lunar" in args:
        pref="lunarcrush" 
    elif "gecko" in args:
        pref="geckocoin"
    
#     print(pref)
    
    lunarav=ping() #lunar avaliable
    geckoav=ping('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=bitcoin') #gecko avaliable
    
    if lunarav==geckoav==False: #if both are down
        bot.send_message(user, "Neither geckocoin nor lunarcrush could be pinged")
        return
    if pref=="lunarcrush" and not lunarav: #if lunar is down, check gecko instead
        pref="geckocoin"
    elif pref=="geckocoin" and not geckoav: #if gecko is down, check lunar instead
        pref="lunarcrush"
        
    #otherwise the preference will be the same
    
#     print(pref)
    
    if pref=="geckocoin": #should've probably made it a separate command
#         print('gecko')
        try:
            url = ("https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=" + 
                   coinstonames[coin.lower()].lower())
            my_json = urllib.request.urlopen(url).read().decode('utf8').replace("'", '"')
            data=json.loads(my_json)[0]
            out=f"{coin} value: {data['current_price']}, checked: {now()}"
            for arg in reqargs:
                if arg in data:
                    out+=f", {arg}: {data[arg]}"
            out+='\nData from geckocoin'
            bot.send_message(user, out)
            return
        except: #geckocoin might not have it, in that case check lunarcrush
            pass
    try: #yeah a separate command would've been easier
        global api_key
        url = "https://api.lunarcrush.com/v2?data=assets&key=" + api_key + "&symbol=" + coin
        assets = json.loads(urllib.request.urlopen(url).read())
        data=assets['data'][0]
        out=f"{coin} value: {data['price']}, checked: {now()}"
        for arg in reqargs:
            if arg in data:
                out+=f", {arg}: {data[arg]}"
        out+='\nData from lunarcrush'
        bot.send_message(user, out)
#         del url, assets, data ###obsolete 
        return
    except:
        bot.send_message(user, nocoin)
        return

In [ ]:
@bot.message_handler(content_types='text')
def gettext(message):
#     print(message.from_user.id)
    inmessage=message.text
    args=list(map(lambda x: str(x).lower(), inmessage.split()))
    command=args[0] #probably not the best way of doing it but definitely the easiest
    args=args[1:]
    sender_id=message.from_user.id
    
    if command=='/start':
        start_bot(sender_id)
        return
    elif command=='/help':
        help_bot(sender_id)
        return
    elif command=='/random':
        random_num(sender_id)
    elif command=='/crypto_args':
        crypto_args(sender_id)
    elif command=="/bugreport":
        bug_rep(" ".join(args))
    elif command=="/crypto":
        crypto_bot(sender_id, args)
    elif command=="/registertracks":
        registertracks(sender_id, args)
    elif command=="/addtracks":
        addtracks(sender_id, args)
    elif command=="/removetracks":
        removetracks(sender_id, args)
    elif command=="/updates":
        track_updates(sender_id, args)
    elif command=="/gettracks":
        gettracks(sender_id)
    elif command=="/nametocrypto":
        nametocoin(sender_id, args)
    elif command=="/cryptotoname":
        cointoname(sender_id, args)
    elif command=="/getall":
        getall(sender_id, args)
    elif command=="/prefered":
        setpreference(sender_id, args)
    else:
        bot.send_message(sender_id, sorrymessage)

In [ ]:
def kbu(wait_time = 120): #keep bot up
    while True:
        try:
            print(f"restarting kbu at {now()}")
            bot.polling(none_stop=True, interval=0)
        except Exception as e:
            print(f"Exception: {e} at {now()}")
            time.sleep(wait_time)

In [ ]:
kbu()
# bot.polling(none_stop=True, interval=0)